In [18]:
import boto3
import io
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
## Setup boto3
s3 = boto3.client('s3')

## Paths for all necessary CSV files
bucket = "cs543-final-project"
patient_data_key = "physionet.org/files/mimic-cxr-jpg/2.0.0/patients.csv"
admissions_data_key = "physionet.org/files/mimic-cxr-jpg/2.0.0/admissions.csv"
outcomes_key = "physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-negbio.csv"
image_data_key = "physionet.org/files/mimic-cxr-jpg/2.0.0/mimic-cxr-2.0.0-metadata.csv"

## Read the csv files
obj = s3.get_object(Bucket=bucket, Key=patient_data_key)
patient_data = pd.read_csv(io.BytesIO(obj['Body'].read()))

obj = s3.get_object(Bucket=bucket, Key=admissions_data_key)
admissions_data = pd.read_csv(io.BytesIO(obj['Body'].read()))

obj = s3.get_object(Bucket=bucket, Key=outcomes_key)
outcome_data = pd.read_csv(io.BytesIO(obj['Body'].read()))

obj = s3.get_object(Bucket=bucket, Key=image_data_key)
image_data = pd.read_csv(io.BytesIO(obj['Body'].read()))

In [34]:
patient_data = patient_data[['subject_id', 'gender', 'anchor_age']]
admissions_data = admissions_data[['subject_id', 'insurance', 'ethnicity']]
image_data = image_data[['subject_id', 'study_id', 'dicom_id']]

In [38]:
df = patient_data.merge(admissions_data, on='subject_id').merge(image_data, on='subject_id').merge(outcome_data, on=['subject_id','study_id'])
df = df[['subject_id','study_id','dicom_id', 'gender', 'anchor_age', 'insurance', 'ethnicity',
        'Atelectasis', 'Cardiomegaly', 'Consolidation',
       'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion',
       'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other',
       'Pneumonia', 'Pneumothorax', 'Support Devices']]

In [43]:
df.to_csv('data/master_df.csv')